# VideoMAE 8-Frame: Combined Mixup + Label Smoothing (Single Phase)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

In [ ]:
!pip install -q transformers accelerate

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor
from transformers import get_cosine_schedule_with_warmup
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import gc

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

PATH_DATA_TRAIN = '/kaggle/input/action-video/data/data_train'
PATH_DATA_TEST = '/kaggle/input/action-video/data/test'

In [ ]:
!gdown "1Xv2CWOqdBj3kt0rkNJKRsodSIEd3-wX_" -O test_labels.csv -q
gt_df = pd.read_csv("test_labels.csv")
TEST_LABELS = dict(zip(gt_df['id'].astype(str), gt_df['class']))
del gt_df; gc.collect()
print(f"Test labels: {len(TEST_LABELS)}")

## 1. Config

In [ ]:
MODEL_CKPT = "MCG-NJU/videomae-base-finetuned-kinetics"
NUM_FRAMES = 8
IMG_SIZE = 224
RESIZE_SIZE = 256

# Single Phase Config
EPOCHS = 60
LR = 5e-5
LABEL_SMOOTHING = 0.1

BATCH_SIZE = 42
NUM_WORKERS = 4
WEIGHT_DECAY = 0.05
MIXUP_ALPHA = 0.8

print(f"Combined: Mixup(alpha={MIXUP_ALPHA}) + LabelSmoothing({LABEL_SMOOTHING})")
print(f"LR={LR}, BATCH={BATCH_SIZE}, EPOCHS={EPOCHS}")

## 2. Dataset

In [ ]:
image_processor = VideoMAEImageProcessor.from_pretrained(MODEL_CKPT)
MEAN, STD = image_processor.image_mean, image_processor.image_std

class MixupCollateWithSmoothing:
    """Mixup + Label Smoothing combined"""
    def __init__(self, num_classes, alpha=0.8, smoothing=0.1):
        self.num_classes = num_classes
        self.alpha = alpha
        self.smoothing = smoothing
        
    def __call__(self, batch):
        inputs, targets = torch.utils.data.default_collate(batch)
        
        # Mixup
        lam = np.random.beta(self.alpha, self.alpha)
        idx = torch.randperm(inputs.size(0))
        inputs = lam * inputs + (1 - lam) * inputs[idx]
        
        # One-hot with Label Smoothing
        oh1 = F.one_hot(targets, self.num_classes).float()
        oh2 = F.one_hot(targets[idx], self.num_classes).float()
        
        # Apply label smoothing to both
        oh1 = oh1 * (1 - self.smoothing) + self.smoothing / self.num_classes
        oh2 = oh2 * (1 - self.smoothing) + self.smoothing / self.num_classes
        
        # Mixup the smoothed labels
        mixed_targets = lam * oh1 + (1 - lam) * oh2
        
        return inputs, mixed_targets

class VideoDataset(Dataset):
    def __init__(self, root, num_frames=8, is_train=True):
        self.root, self.num_frames, self.is_train = Path(root), num_frames, is_train
        self.classes = sorted([d.name for d in self.root.iterdir() if d.is_dir()])
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.samples = [(list(sorted(v.glob('*.jpg'))), self.class_to_idx[c]) 
                        for c in self.classes for v in (self.root/c).iterdir() if v.is_dir()]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        paths, label = self.samples[idx]
        indices = np.linspace(0, len(paths)-1, self.num_frames, dtype=int)
        frames = [TF.resize(Image.open(paths[i]).convert('RGB'), RESIZE_SIZE) for i in indices]
        if self.is_train:
            i,j,h,w = T.RandomResizedCrop.get_params(frames[0], (0.8,1.0), (0.75,1.33))
            flip = random.random() > 0.5
            frames = [TF.normalize(TF.to_tensor(TF.hflip(TF.resized_crop(f,i,j,h,w,(IMG_SIZE,IMG_SIZE))) if flip else TF.resized_crop(f,i,j,h,w,(IMG_SIZE,IMG_SIZE))), MEAN, STD) for f in frames]
        else:
            frames = [TF.normalize(TF.to_tensor(TF.center_crop(f, IMG_SIZE)), MEAN, STD) for f in frames]
        return torch.stack(frames), label

class TestDataset(Dataset):
    def __init__(self, root, num_frames=8):
        self.root, self.num_frames = Path(root), num_frames
        self.videos = sorted([d for d in self.root.iterdir() if d.is_dir()], key=lambda x: int(x.name))
    def __len__(self): return len(self.videos)
    def __getitem__(self, idx):
        v = self.videos[idx]
        paths = sorted(v.glob('*.jpg'))
        indices = np.linspace(0, len(paths)-1, self.num_frames, dtype=int)
        frames = [TF.normalize(TF.to_tensor(TF.center_crop(TF.resize(Image.open(paths[i]).convert('RGB'), RESIZE_SIZE), IMG_SIZE)), MEAN, STD) for i in indices]
        return torch.stack(frames), int(v.name)

In [ ]:
train_dataset = VideoDataset(PATH_DATA_TRAIN, NUM_FRAMES, is_train=True)
test_dataset = TestDataset(PATH_DATA_TEST, NUM_FRAMES)
print(f"Train: {len(train_dataset)} | Test: {len(test_dataset)} | Classes: {len(train_dataset.classes)}")

# Combined collate function
combined_collate = MixupCollateWithSmoothing(len(train_dataset.classes), MIXUP_ALPHA, LABEL_SMOOTHING)
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=combined_collate, drop_last=True)
test_loader = DataLoader(test_dataset, BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=True)

## 3. Model

In [ ]:
model = VideoMAEForVideoClassification.from_pretrained(
    MODEL_CKPT, num_labels=len(train_dataset.classes), 
    ignore_mismatched_sizes=True, num_frames=NUM_FRAMES
).to(DEVICE)
print(f"Params: {sum(p.numel() for p in model.parameters()):,}")

## 4. Training Functions

In [ ]:
def train_epoch(model, loader, optimizer, scheduler, scaler):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    
    for x, y in tqdm(loader, desc="Train", leave=False):
        x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
        
        with torch.amp.autocast('cuda'):
            logits = model(x).logits
            # Soft cross entropy (for mixed + smoothed labels)
            loss = -torch.sum(y * F.log_softmax(logits, 1), 1).mean()
            labels = y.argmax(1)
        
        total_loss += loss.item()
        correct += (logits.argmax(1) == labels).sum().item()
        total += x.size(0)
        
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        del x, y, logits, loss
    
    return total_loss / len(loader), correct / total

@torch.no_grad()
def evaluate(model, loader, classes):
    model.eval()
    all_preds, all_ids = [], []
    
    for x, ids in tqdm(loader, desc="Eval", leave=False):
        x = x.to(DEVICE, non_blocking=True)
        preds = model(x).logits.argmax(1).cpu().numpy()
        all_preds.extend(preds.tolist())
        all_ids.extend(ids.numpy().tolist())
        del x, preds
    
    y_pred = [classes[p] for vid, p in zip(all_ids, all_preds) if str(vid) in TEST_LABELS]
    y_true = [TEST_LABELS[str(vid)] for vid in all_ids if str(vid) in TEST_LABELS]
    return accuracy_score(y_true, y_pred)

def save_checkpoint(epoch, loss, train_acc, test_acc, best_acc):
    torch.save(model.state_dict(), 'checkpoint.pt')
    row = {'epoch': epoch, 'loss': loss, 'train_acc': train_acc, 'test_acc': test_acc, 'best_acc': best_acc}
    df = pd.DataFrame([row])
    if os.path.exists('history.csv'):
        df.to_csv('history.csv', mode='a', header=False, index=False)
    else:
        df.to_csv('history.csv', index=False)
    print(f"  Loss:{loss:.4f} | Train:{train_acc:.4f} | Test:{test_acc:.4f} | Best:{best_acc:.4f}")

## 5. Check Resume

In [ ]:
START_EPOCH = 1
best_acc = 0

if os.path.exists('history.csv') and os.path.exists('checkpoint.pt'):
    df = pd.read_csv('history.csv')
    last = df.iloc[-1]
    model.load_state_dict(torch.load('checkpoint.pt', map_location=DEVICE))
    START_EPOCH = int(last['epoch']) + 1
    best_acc = float(last['best_acc'])
    print(f"✅ Resume from epoch {START_EPOCH}, best={best_acc:.4f}")
else:
    print("📝 Starting fresh")

## 6. Training (Single Phase: Mixup + Label Smoothing)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
total_steps = len(train_loader) * EPOCHS
scheduler = get_cosine_schedule_with_warmup(optimizer, int(total_steps*0.1), total_steps)

# Fast-forward scheduler if resuming
for _ in range((START_EPOCH - 1) * len(train_loader)):
    scheduler.step()

scaler = torch.amp.GradScaler()

print("="*60)
print(f"SINGLE PHASE: Mixup + LabelSmoothing | LR={LR} | Epochs {START_EPOCH}-{EPOCHS}")
print("="*60)

for ep in range(START_EPOCH, EPOCHS + 1):
    print(f"\nEpoch {ep}/{EPOCHS}")
    loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, scaler)
    test_acc = evaluate(model, test_loader, train_dataset.classes)
    
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save({'model': model.state_dict(), 'classes': train_dataset.classes}, 'best.pt')
        print(f"  >>> New Best!")
    
    save_checkpoint(ep, loss, train_acc, test_acc, best_acc)
    gc.collect(); torch.cuda.empty_cache()

print(f"\n🏆 FINAL BEST: {best_acc:.4f}")

## 7. Plot

In [ ]:
if os.path.exists('history.csv'):
    df = pd.read_csv('history.csv')
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(df['epoch'], df['train_acc'], 'b-o', label='Train', ms=3)
    axes[0].plot(df['epoch'], df['test_acc'], 'r-s', label='Test', ms=3)
    axes[0].set_title('Accuracy (Combined Mixup+LabelSmoothing)'); axes[0].legend(); axes[0].grid(alpha=0.3)
    axes[1].plot(df['epoch'], df['loss'], 'g-^', ms=3)
    axes[1].set_title('Loss'); axes[1].grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig('curves.png', dpi=150)
    plt.show()
    print(df.to_string(index=False))